# Python example

We will test a simple Pipeline with a cifar10 image classification model using a large batch size. We assume a locally running Seldon.

In [ ]:
import requests
import json
from typing import Dict, List
import numpy as np
import os
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
train, test = tf.keras.datasets.cifar10.load_data()
X_train, y_train = train
X_test, y_test = test

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
!cat ./models/cifar10.yaml

In [ ]:
!seldon model load -f ./models/cifar10.yaml

In [ ]:
!seldon model status --model-name cifar10 -w ModelAvailable | jq .

In [ ]:
!cat ./pipelines/cifar10.yaml

In [ ]:
!seldon pipeline load -f ./pipelines/cifar10.yaml

In [ ]:
!seldon pipeline status -p cifar10-test -w PipelineReady| jq -M .

In [ ]:
headers = {"Content-Type": "application/json", "seldon-model":"cifar10-test.pipeline"}
reqJson = json.loads('{"inputs":[{"name":"input_1","data":[],"datatype":"FP32","shape":[]}]}')
url = "http://0.0.0.0:9000/v2/models/cifar10/infer"

In [ ]:
batchSz = 1000
rows = X_train[0:0+batchSz]
reqJson["inputs"][0]["data"] = rows.flatten().tolist()
reqJson["inputs"][0]["shape"] = [batchSz, 32, 32, 3]

In [ ]:
response_raw = requests.post(url, json=reqJson, headers=headers)
print(response_raw)
print(response_raw.json())

In [ ]:
!seldon pipeline unload -p cifar10-test
!seldon model unload --model-name cifar10